In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
0.将 预测结果保持小数形式，而不是转换成整数形式  结果从0.5以上优化到0.5以下 0.47381812389478345
1.将 survey_time数据提取出月份、日期、小时三个特征 0.47143198583365375
2.将 family_income，income离散成5段的离散值 0.4730428161760329
3.将 birth 转换成年龄值age 0.47143198583365375
4.将 family_income，income离散成5段的离散值，并且birth 转换成年龄值age，都加上，0.47241532744595904
6.将 age 离散化成7段年龄段  0.47619062169156706
7.将 survey_time数据提取出一周的第几天、是否周末 两个特征 0.47097871771026456
8.将 family_income、income中小于0的数字转换为0 0.4708401593321734
9.使用全量特征训练 0.466104881012437
10.将 family_income，income提取出来家庭收入占个人收入的比例特征，0.46458776964941884
11.调参：
    1.将max_depth由6调整为8，0.46458776964941884-》0.4623970704927228
    2.将n_estimators由100调整为200  0.4623970704927228-》0.462396775672775
    3.将min_child_weight由1-》2-》3  0.462396775672775-》0.4616176256431441-》0.46051453956155897
    4.将subsample由1调整为0.6  0.46051453956155897-》0.4552454001761164
    5.将colsample_bytree由1调整为0.6  0.4552454001761164->0.4520618951047676
"""

import xgboost as xgb
from xgboost import plot_importance
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import preprocessing 
%matplotlib inline

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_complete.csv'
test_data_file = 'data/happiness_test_complete.csv'


In [3]:
df_train = pd.read_csv(data_dir+train_data_file, encoding='gb2312')
df_test = pd.read_csv(data_dir+test_data_file, encoding='gb2312')
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,4,50,60,50,50,30.0,30,50,50,50
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,3,90,70,70,80,85.0,70,90,60,60
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,4,90,80,75,79,80.0,90,90,90,75
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,3,100,90,70,80,80.0,90,90,80,80
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,2,50,50,50,50,50.0,50,50,50,50


In [4]:
df_train = df_train.drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train[pd.to_datetime(df_train.survey_time).dt.month != 2]


In [5]:
# survey_time数据提取出月份、日期、小时三个特征
df_train['month'] = pd.to_datetime(df_train.survey_time).dt.month
df_train['day'] = pd.to_datetime(df_train.survey_time).dt.day
df_train['hour'] = pd.to_datetime(df_train.survey_time).dt.hour
df_train['dayofweek'] = pd.to_datetime(df_train.survey_time).dt.dayofweek+1
df_train['is_weekend'] = (pd.to_datetime(df_train.survey_time).dt.dayofweek+1).apply(lambda x: 1 if x in (6, 7) else 0)
df_test['month'] = pd.to_datetime(df_test.survey_time).dt.month
df_test['day'] = pd.to_datetime(df_test.survey_time).dt.day
df_test['hour'] = pd.to_datetime(df_test.survey_time).dt.hour
df_test['dayofweek'] = pd.to_datetime(df_test.survey_time).dt.dayofweek+1
df_test['is_weekend'] = (pd.to_datetime(df_test.survey_time).dt.dayofweek+1).apply(lambda x: 1 if x in (6, 7) else 0)
# 去除 survey_time数据
df_train = df_train.drop('survey_time', 1)
df_test = df_test.drop('survey_time', 1)
# 家庭收入是个人收入的比例
df_train['income_rate'] = df_train.apply(lambda row: row['family_income']/row['income'] if row['family_income']*row['income']>0 else 0, 1)
df_test['income_rate'] = df_test.apply(lambda row: row['family_income']/row['income'] if row['family_income']*row['income']>0 else 0, 1)
# 将family_income、income小于0的值转换为0值
df_train['family_income'] = df_train['family_income'].apply(lambda x: x if x>0 else 0)
df_train['income'] = df_train['income'].apply(lambda x: x if x>0 else 0)
df_test['family_income'] = df_test['family_income'].apply(lambda x: x if x>0 else 0)
df_test['income'] = df_test['income'].apply(lambda x: x if x>0 else 0)
# family_income，income离散成5段的离散值
family_income_quantile = df_train.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_train['family_income_stage'] = pd.cut(df_train.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])
income_quantile = df_train.income.quantile([0, 0.33, 0.66, 1]).values
df_train['income_stage'] = pd.cut(df_train.income, income_quantile, labels=[0, 1, 2])
family_income_quantile = df_test.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_test['family_income_stage'] = pd.cut(df_test.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])
income_quantile = df_test.income.quantile([0, 0.33, 0.66, 1]).values
df_test['income_stage'] = pd.cut(df_test.income, income_quantile, labels=[0, 1, 2])
# birth 转换成年龄值
df_train['age']  = (2019 - df_train.birth)
df_train = df_train.drop('birth', 1)
# df_train['age'] = pd.cut(df_train.age, [0, 20, 30, 40, 50, 60, 70, 100], labels=[0, 1, 2, 3, 4, 5, 6])
df_test['age']  = (2019 - df_test.birth)
df_test = df_test.drop('birth', 1)
# df_test['age'] = pd.cut(df_test.age, [0, 20, 30, 40, 50, 60, 70, 100], labels=[0, 1, 2, 3, 4, 5, 6])


df_train.head()


,happiness,survey_type,province,city,county,gender,nationality,religion,religion_freq,edu,...,public_service_9,month,day,hour,dayofweek,is_weekend,income_rate,family_income_stage,income_stage,age
0,4,1,12,32,59,1,1,1,1,11,...,50,8,4,14,2,0,3.000000,3,1,60
1,4,2,18,52,85,1,1,1,1,12,...,60,7,21,15,2,0,2.000000,2,1,27
2,4,2,29,83,126,2,1,0,3,4,...,75,7,21,13,2,0,4.000000,0,0,52
3,5,2,10,28,51,2,1,1,1,3,...,80,7,25,17,6,1,1.869159,1,1,76
4,4,1,7,18,36,2,1,1,1,12,...,50,8,10,9,1,0,2.000000,NaN,NaN,25


In [6]:
df_train = df_train.drop('property_other', 1).drop('edu_other', 1).drop('invest_other', 1)
df_test = df_test.drop('property_other', 1).drop('edu_other', 1).drop('invest_other', 1)

In [7]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]

In [8]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()

scaler = preprocessing.StandardScaler().fit(tr_data)
tr_data = scaler.transform(tr_data)

scaler = preprocessing.StandardScaler().fit(va_data)
va_data = scaler.transform(va_data)

scaler = preprocessing.StandardScaler().fit(te_data)
te_data = scaler.transform(te_data)

# tr_x_data = tr_data[:, 1:]
# tr_y_data = tr_data[:, 0]

# va_x_data = va_data[:, 1:]
# va_y_data = va_data[:, 0]

/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:

batch_size = 128

def get_batch_data(data):
    tr_shu = np.arange(len(data))
    np.random.shuffle(tr_data)
    tr_batch = data[tr_shu[:batch_size]]
    
    return tr_batch[:, 1:], np.reshape(tr_batch[:, 0], [batch_size, 1])



In [19]:
x = tf.placeholder(tf.float32, shape=[None, 142], name='x')
y_label = tf.placeholder(tf.float32, shape=[None, 1], name='y_label')

# W1 = tf.Variable(tf.zeros([142,64]), name='W1')
# b1 = tf.Variable(tf.zeros([64]), name='b1')
# W2 = tf.Variable(tf.zeros([64,64]), name='W2')
# b2 = tf.Variable(tf.zeros([64]), name='b2')
# W3 = tf.Variable(tf.zeros([64,1]), name='W3')
# b3 = tf.Variable(tf.zeros([1]), name='b3')

# regularizer = tf.contrib.layers.l2_regularizer(0.01)
# wloss1 = regularizer(W1)
# wloss2 = regularizer(W2)
# wloss3 = regularizer(W3)
# tf.add_to_collection('losses', wloss1)
# tf.add_to_collection('losses', wloss2)
# tf.add_to_collection('losses', wloss3)


# h1 = tf.add(tf.matmul(x, W1), b1)
# h2 = tf.add(tf.matmul(h1, W2), b2)
# y_pred = tf.add(tf.matmul(h2, W3), b3)

W = tf.Variable(tf.zeros([142,1]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')
y_pred = tf.add(tf.matmul(x, W), b)

cost = tf.reduce_mean(tf.square(y_label - y_pred))
# tf.add_to_collection('losses', cost)
# loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
# optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.001).minimize(cost)






cost_summary = tf.summary.scalar('Cost', cost)
merged = tf.summary.merge_all()

n = 100000
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    writer = tf.summary.FileWriter('.', sess.graph)

    for i in range(n):
        x_train_batch, y_train_batch = get_batch_data(tr_data)
        x_va_batch, y_va_batch = get_batch_data(va_data)
        
        _, pred = sess.run([optimizer, y_pred], feed_dict={x: x_train_batch, y_label: y_train_batch}) 
        err = sess.run(cost, feed_dict={x: x_va_batch, y_label: y_va_batch})
        print("%d batch error %.4f:" % (i, err))
        


0 batch error 0.8346:
1 batch error 0.8323:
2 batch error 0.8267:
3 batch error 0.8220:
4 batch error 0.8183:
5 batch error 0.8146:
6 batch error 0.8104:
7 batch error 0.8087:
8 batch error 0.8055:
9 batch error 0.8013:
10 batch error 0.7985:
11 batch error 0.7949:
12 batch error 0.7905:
13 batch error 0.7889:
14 batch error 0.7879:
15 batch error 0.7865:
16 batch error 0.7847:
17 batch error 0.7802:
18 batch error 0.7774:
19 batch error 0.7736:
20 batch error 0.7704:
21 batch error 0.7640:
22 batch error 0.7611:
23 batch error 0.7544:
24 batch error 0.7498:
25 batch error 0.7476:
26 batch error 0.7452:
27 batch error 0.7416:
28 batch error 0.7353:
29 batch error 0.7336:
30 batch error 0.7336:
31 batch error 0.7305:
32 batch error 0.7262:
33 batch error 0.7232:
34 batch error 0.7204:
35 batch error 0.7174:
36 batch error 0.7159:
37 batch error 0.7155:
38 batch error 0.7126:
39 batch error 0.7127:
40 batch error 0.7081:
41 batch error 0.7051:
42 batch error 0.7029:
43 batch error 0.7029

348 batch error 0.6455:
349 batch error 0.6466:
350 batch error 0.6457:
351 batch error 0.6440:
352 batch error 0.6440:
353 batch error 0.6396:
354 batch error 0.6401:
355 batch error 0.6420:
356 batch error 0.6398:
357 batch error 0.6375:
358 batch error 0.6370:
359 batch error 0.6357:
360 batch error 0.6363:
361 batch error 0.6403:
362 batch error 0.6406:
363 batch error 0.6439:
364 batch error 0.6444:
365 batch error 0.6422:
366 batch error 0.6412:
367 batch error 0.6401:
368 batch error 0.6394:
369 batch error 0.6389:
370 batch error 0.6396:
371 batch error 0.6364:
372 batch error 0.6384:
373 batch error 0.6375:
374 batch error 0.6373:
375 batch error 0.6362:
376 batch error 0.6351:
377 batch error 0.6336:
378 batch error 0.6345:
379 batch error 0.6345:
380 batch error 0.6364:
381 batch error 0.6361:
382 batch error 0.6348:
383 batch error 0.6341:
384 batch error 0.6355:
385 batch error 0.6370:
386 batch error 0.6372:
387 batch error 0.6342:
388 batch error 0.6316:
389 batch error 

699 batch error 0.6203:
700 batch error 0.6200:
701 batch error 0.6224:
702 batch error 0.6238:
703 batch error 0.6229:
704 batch error 0.6204:
705 batch error 0.6198:
706 batch error 0.6158:
707 batch error 0.6178:
708 batch error 0.6184:
709 batch error 0.6172:
710 batch error 0.6208:
711 batch error 0.6237:
712 batch error 0.6239:
713 batch error 0.6226:
714 batch error 0.6225:
715 batch error 0.6213:
716 batch error 0.6205:
717 batch error 0.6227:
718 batch error 0.6243:
719 batch error 0.6242:
720 batch error 0.6239:
721 batch error 0.6220:
722 batch error 0.6208:
723 batch error 0.6217:
724 batch error 0.6220:
725 batch error 0.6189:
726 batch error 0.6187:
727 batch error 0.6201:
728 batch error 0.6203:
729 batch error 0.6190:
730 batch error 0.6230:
731 batch error 0.6238:
732 batch error 0.6232:
733 batch error 0.6241:
734 batch error 0.6282:
735 batch error 0.6267:
736 batch error 0.6247:
737 batch error 0.6230:
738 batch error 0.6261:
739 batch error 0.6261:
740 batch error 

1040 batch error 0.6392:
1041 batch error 0.6414:
1042 batch error 0.6406:
1043 batch error 0.6406:
1044 batch error 0.6402:
1045 batch error 0.6377:
1046 batch error 0.6406:
1047 batch error 0.6399:
1048 batch error 0.6386:
1049 batch error 0.6388:
1050 batch error 0.6409:
1051 batch error 0.6401:
1052 batch error 0.6350:
1053 batch error 0.6361:
1054 batch error 0.6363:
1055 batch error 0.6383:
1056 batch error 0.6377:
1057 batch error 0.6365:
1058 batch error 0.6372:
1059 batch error 0.6382:
1060 batch error 0.6416:
1061 batch error 0.6448:
1062 batch error 0.6457:
1063 batch error 0.6485:
1064 batch error 0.6461:
1065 batch error 0.6496:
1066 batch error 0.6485:
1067 batch error 0.6475:
1068 batch error 0.6470:
1069 batch error 0.6473:
1070 batch error 0.6461:
1071 batch error 0.6452:
1072 batch error 0.6424:
1073 batch error 0.6396:
1074 batch error 0.6400:
1075 batch error 0.6440:
1076 batch error 0.6443:
1077 batch error 0.6467:
1078 batch error 0.6444:
1079 batch error 0.6411:


1374 batch error 0.6273:
1375 batch error 0.6292:
1376 batch error 0.6287:
1377 batch error 0.6307:
1378 batch error 0.6309:
1379 batch error 0.6288:
1380 batch error 0.6301:
1381 batch error 0.6302:
1382 batch error 0.6302:
1383 batch error 0.6319:
1384 batch error 0.6351:
1385 batch error 0.6271:
1386 batch error 0.6261:
1387 batch error 0.6275:
1388 batch error 0.6276:
1389 batch error 0.6278:
1390 batch error 0.6304:
1391 batch error 0.6317:
1392 batch error 0.6327:
1393 batch error 0.6276:
1394 batch error 0.6285:
1395 batch error 0.6293:
1396 batch error 0.6321:
1397 batch error 0.6314:
1398 batch error 0.6316:
1399 batch error 0.6296:
1400 batch error 0.6288:
1401 batch error 0.6264:
1402 batch error 0.6268:
1403 batch error 0.6267:
1404 batch error 0.6270:
1405 batch error 0.6248:
1406 batch error 0.6262:
1407 batch error 0.6253:
1408 batch error 0.6222:
1409 batch error 0.6255:
1410 batch error 0.6268:
1411 batch error 0.6292:
1412 batch error 0.6275:
1413 batch error 0.6269:


...

98698 batch error 0.6233:
98699 batch error 0.6278:
98700 batch error 0.6249:
98701 batch error 0.6257:
98702 batch error 0.6232:
98703 batch error 0.6232:
98704 batch error 0.6266:
98705 batch error 0.6273:
98706 batch error 0.6243:
98707 batch error 0.6256:
98708 batch error 0.6265:
98709 batch error 0.6286:
98710 batch error 0.6315:
98711 batch error 0.6284:
98712 batch error 0.6259:
98713 batch error 0.6271:
98714 batch error 0.6243:
98715 batch error 0.6252:
98716 batch error 0.6257:
98717 batch error 0.6234:
98718 batch error 0.6239:
98719 batch error 0.6231:
98720 batch error 0.6250:
98721 batch error 0.6245:
98722 batch error 0.6248:
98723 batch error 0.6229:
98724 batch error 0.6231:
98725 batch error 0.6219:
98726 batch error 0.6235:
98727 batch error 0.6246:
98728 batch error 0.6241:
98729 batch error 0.6242:
98730 batch error 0.6228:
98731 batch error 0.6204:
98732 batch error 0.6184:
98733 batch error 0.6182:
98734 batch error 0.6208:
98735 batch error 0.6179:
98736 batch 

99018 batch error 0.6087:
99019 batch error 0.6070:
99020 batch error 0.6074:
99021 batch error 0.6061:
99022 batch error 0.6072:
99023 batch error 0.6114:
99024 batch error 0.6131:
99025 batch error 0.6131:
99026 batch error 0.6166:
99027 batch error 0.6154:
99028 batch error 0.6172:
99029 batch error 0.6157:
99030 batch error 0.6157:
99031 batch error 0.6153:
99032 batch error 0.6154:
99033 batch error 0.6160:
99034 batch error 0.6168:
99035 batch error 0.6136:
99036 batch error 0.6131:
99037 batch error 0.6134:
99038 batch error 0.6148:
99039 batch error 0.6152:
99040 batch error 0.6128:
99041 batch error 0.6150:
99042 batch error 0.6158:
99043 batch error 0.6149:
99044 batch error 0.6166:
99045 batch error 0.6126:
99046 batch error 0.6122:
99047 batch error 0.6132:
99048 batch error 0.6141:
99049 batch error 0.6151:
99050 batch error 0.6178:
99051 batch error 0.6194:
99052 batch error 0.6195:
99053 batch error 0.6211:
99054 batch error 0.6209:
99055 batch error 0.6240:
99056 batch 

99343 batch error 0.6134:
99344 batch error 0.6140:
99345 batch error 0.6140:
99346 batch error 0.6141:
99347 batch error 0.6148:
99348 batch error 0.6145:
99349 batch error 0.6135:
99350 batch error 0.6158:
99351 batch error 0.6147:
99352 batch error 0.6147:
99353 batch error 0.6135:
99354 batch error 0.6148:
99355 batch error 0.6172:
99356 batch error 0.6178:
99357 batch error 0.6143:
99358 batch error 0.6148:
99359 batch error 0.6147:
99360 batch error 0.6156:
99361 batch error 0.6149:
99362 batch error 0.6161:
99363 batch error 0.6166:
99364 batch error 0.6161:
99365 batch error 0.6167:
99366 batch error 0.6167:
99367 batch error 0.6179:
99368 batch error 0.6185:
99369 batch error 0.6201:
99370 batch error 0.6205:
99371 batch error 0.6203:
99372 batch error 0.6201:
99373 batch error 0.6215:
99374 batch error 0.6188:
99375 batch error 0.6179:
99376 batch error 0.6199:
99377 batch error 0.6191:
99378 batch error 0.6198:
99379 batch error 0.6184:
99380 batch error 0.6219:
99381 batch 

99664 batch error 0.6250:
99665 batch error 0.6280:
99666 batch error 0.6284:
99667 batch error 0.6243:
99668 batch error 0.6217:
99669 batch error 0.6254:
99670 batch error 0.6260:
99671 batch error 0.6251:
99672 batch error 0.6278:
99673 batch error 0.6229:
99674 batch error 0.6234:
99675 batch error 0.6242:
99676 batch error 0.6244:
99677 batch error 0.6249:
99678 batch error 0.6238:
99679 batch error 0.6243:
99680 batch error 0.6229:
99681 batch error 0.6221:
99682 batch error 0.6236:
99683 batch error 0.6220:
99684 batch error 0.6205:
99685 batch error 0.6217:
99686 batch error 0.6199:
99687 batch error 0.6197:
99688 batch error 0.6194:
99689 batch error 0.6185:
99690 batch error 0.6187:
99691 batch error 0.6177:
99692 batch error 0.6176:
99693 batch error 0.6153:
99694 batch error 0.6161:
99695 batch error 0.6162:
99696 batch error 0.6119:
99697 batch error 0.6142:
99698 batch error 0.6172:
99699 batch error 0.6178:
99700 batch error 0.6206:
99701 batch error 0.6200:
99702 batch 

99988 batch error 0.6267:
99989 batch error 0.6271:
99990 batch error 0.6241:
99991 batch error 0.6246:
99992 batch error 0.6259:
99993 batch error 0.6236:
99994 batch error 0.6243:
99995 batch error 0.6215:
99996 batch error 0.6220:
99997 batch error 0.6204:
99998 batch error 0.6243:
99999 batch error 0.6204:
